<h1 style="text-align: center;">Minería de Datos</h1>

<h2>Tercera y Cuarta Entrega: Clasificadores y Ensamble</h2>

**Alumno:** Daniel Garnica Sánchez

**Índice**

* Tercera Entrega
    * Introducción
        * ¿Qué es un Ensamble?
    * Objetivos
    * Desarrollo
        * Preprocesamiento de Datos
        * Estandarización de Datos
        * PCA
    * Clasificadores de Información
        * Naive Bayes
        * Árbol de Decisión
        * KNN

* Cuarta Entrega
    * Ensamble de Clasificadores
    * Promedio de Predicciones
    * Método de Votación
    * Predicción y Resultados
    * Conclusión

# Introducción



A lo largo de las anteriores entregas se ha ido explorando continuamente diferentes técnicas para la trata de información, tales como el preprocesamiento, donde se prepara la información para ser procesada con métodos como la estandarización de datos, la detección y eliminación de outliers, etc. 


De la misma manera se trabajo con el Análisis de Componentes Principales, el cual constituye un procedimiento matemático que permite transformar un número de variables posiblemente correlacionadas en un número menor de variables no correlacionadas, llamadas: Componentes Principales. Esto con el fin de reducir la dimensionalidad de nuestra información. 


Posteriormente se entrenó un clasificador de tipo Árbol, donde nuestro dataset se dividió en dos conjuntos, uno de entrenamiento y otro de prueba, para posteriormente obtener la precisión del clasificador. 


El siguiente paso es formar un ensamble de clasificadores.

## ¿Qué es un Ensamble?

Como sabemos los clasificadores nos permiten hacer un análisis y una predicción de la clase de nuestra información. Un clasificador utiliza un conjunto de entrenamiento para comprender como los atributos se relacionan con las clases y una vez que ha sido entrenado puede utilizarse para detectar algún nuevo caso o hacer predicciones. 

Un ensamble es un conjunto de clasificadores que nos permite combinar múltiples hipótesis para formar una hipótesis mejor y más precisa. Básicamente los métodos de ensamble son técnicas que nos permite combinar varios algoritmos de aprendizaje para construir un algoritmo más poderoso y por ende más preciso para obtener un mejor desempeño en la predicción.

# Objetivos

El objetivo de esta entrega es mostrar los diferentes tipos clasificadores que se utilizarán para la creación de mi ensamble. 

Anteriormente se había trabajado con el clasificador de Árbol de Decisión por lo que se decidió incluirlo para la creación del ensamble.

# Desarrollo

A continuación se mostrará el proceso para la creación los clasificadores así como su entrenamiento y su precisión conforme al segundo conjunto de datos que se nos brindó, el cual cuenta con muchos más atributos.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

## Preprocesamiento de Datos

Para obtener resultados más satisfactorios en nuestros clasificadores es necesario hacer un preprocesamiento de la información, es por eso que este apartado mostrará el desarrollo de esto, sin embargo no se entrará mucho en detalle, ya que es un proceso que se ha mostrado anteriormente en las demás entregas. 

In [2]:
info = 'data385Attrib.txt'

with open(info, "r") as archivo:
    numElementos = int(archivo.readline())
    numAtributos = int(archivo.readline())
    numClases = int(archivo.readline())
    
    atributos = []
    
    for i in range(0, numAtributos):
        atributos.append("A" + str(i+1))

    atributos.append("Clase")
    
    data = pd.read_csv(archivo, delimiter=',', header=None)
    data.columns = atributos

In [3]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A377,A378,A379,A380,A381,A382,A383,A384,A385,Clase
0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,0.980381,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.803851,0
1,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.745726,0
2,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.687600,0
3,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.629474,0
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,-0.25,...,0.976833,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.571348,0


### Estandarización de Datos

In [4]:
dataNoClase = data.drop('Clase', 1)
y_final = data['Clase']

x_std = StandardScaler().fit_transform(dataNoClase)
atributos.remove('Clase')

x_final = pd.DataFrame(data = x_std, columns = atributos)

In [5]:
x_final.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A376,A377,A378,A379,A380,A381,A382,A383,A384,A385
0,-0.458699,-0.307113,-0.243608,-0.307003,-0.518008,-0.675602,-1.003106,-0.760053,-0.659573,-0.1849,...,-1.346831,2.398449,-0.608699,-0.69581,-0.707798,-0.499451,-0.009384,-1.20185,-0.641154,-0.332086
1,-0.458699,-0.307113,-0.243608,-0.307003,-0.518008,-0.675602,-1.003106,-0.760053,-0.659573,-0.1849,...,-1.346831,2.388748,-0.608699,-0.69581,-0.707798,-0.499451,-0.009384,-1.20185,-0.641154,-0.339097
2,-0.458699,-0.307113,-0.243608,-0.307003,-0.518008,-0.675602,-1.003106,-0.760053,-0.659573,-0.1849,...,-1.346831,2.388748,-0.608699,-0.69581,-0.707798,-0.499451,-0.009384,-1.20185,-0.641154,-0.346107
3,-0.458699,-0.307113,-0.243608,-0.307003,-0.518008,-0.675602,-1.003106,-0.760053,-0.659573,-0.1849,...,-1.346831,2.388748,-0.608699,-0.69581,-0.707798,-0.499451,-0.009384,-1.20185,-0.641154,-0.353118
4,-0.458699,-0.307113,-0.243608,-0.307003,-0.518008,-0.675602,-1.003106,-0.760053,-0.659573,-0.1849,...,-1.346831,2.388245,-0.608699,-0.69581,-0.707798,-0.499451,-0.009384,-1.20185,-0.641154,-0.360128


## PCA

Se optó por utilizar el Análisis de Componentes Principales debido a la gran cantidad de atributos que nuestros elementos tienen, además de proporcionar más precisión a nuestros clasificadores.

In [6]:
num_comp = int(input("Ingrese la cantidad de Componentes Principales: "))
pca = PCA(n_components = num_comp)

Ingrese la cantidad de Componentes Principales: 10


In [7]:
atributos = []
for i in range(num_comp):
    atributos.append('CP'+ str(i+1))

In [8]:
atributos

['CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CP7', 'CP8', 'CP9', 'CP10']

In [9]:
x_pca = pca.fit_transform(x_std)
x_final = pd.DataFrame(data = x_pca, columns = atributos)

In [10]:
x_final.head()

,CP1,CP2,CP3,CP4,CP5,CP6,CP7,CP8,CP9,CP10
0,10.460596,-3.339109,-4.563161,3.327759,-2.208121,6.064103,-4.216890,-2.282480,2.566458,-0.184194
1,10.476073,-3.105652,-4.598310,3.396971,-2.553714,6.859681,-4.514791,-2.525036,2.475475,-0.264140
2,10.461930,-3.080342,-4.655892,3.387952,-2.533418,7.027484,-4.408574,-2.720884,2.408274,-0.350027
3,10.266744,-3.118854,-4.481004,3.101231,-2.716013,6.562618,-3.877002,-2.767032,2.268119,-0.407711
4,10.291467,-3.159250,-4.525222,3.179001,-2.780219,6.549292,-3.847466,-2.891800,2.291335,-0.392369


## Clasificadores de Información

In [11]:
dataFinal = pd.concat([x_final, y_final], axis = 1);
X = x_final
Y = y_final

Para entrenar los diversos clasificadores es nececsario dividir nuestra información ya limpia y estandarizada en dos conjuntos:

- Conjunto de entrenamiento
- Conjunto de prueba

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)

## Naive Bayes

Naive Bayes es un clasificador probabilístico inspirado en el teorema de Bayes bajo una simple suposición de que los atributos son condicionalmente independientes.

Naive Bayes es un algoritmo muy simple de implementar y se han obtenido buenos resultados en la mayoría de los casos. Puede ser fácilmente escalable a conjuntos de datos más grandes, ya que lleva tiempo lineal, en lugar de una aproximación iterativa costosa como se usa para muchos otros tipos de clasificadores.

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf_bayes = GaussianNB().fit(X_train, y_train)
bayes_pred = clf_bayes.predict(X_test)

### Reporte de Clasificación

In [14]:
print(classification_report(y_test, bayes_pred))

              precision    recall  f1-score   support

           0       0.73      0.47      0.57        17
           1       0.88      0.82      0.85        51
           2       0.70      0.70      0.70        10
           3       1.00      1.00      1.00         5
           4       0.42      0.67      0.52        12
           5       0.75      0.81      0.78        26

    accuracy                           0.75       121
   macro avg       0.75      0.74      0.74       121
weighted avg       0.77      0.75      0.76       121



### Matríz de Confusión 

In [15]:
print(confusion_matrix(y_test, bayes_pred))

[[ 8  4  0  0  4  1]
 [ 0 42  3  0  1  5]
 [ 1  1  7  0  1  0]
 [ 0  0  0  5  0  0]
 [ 2  1  0  0  8  1]
 [ 0  0  0  0  5 21]]


### Precisión de Clasificador Naive Bayes

In [16]:
print(accuracy_score(y_test, bayes_pred))

0.7520661157024794


Podemos notar como la precisión de nuestro clasificador no es del todo optima, sin embargo veremos como se compara esta precisión con la de los demás clasificadores.

## Árbol de Decisión

El árbol de decisión crea modelos de clasificación en forma de estructura de árbol. Utiliza un conjunto de reglas las cuales son exclusivas y exhaustivas para la clasificación.


Este tipo de clasificador puede ajustarse fácilmente generando demasiadas ramas y puede reflejar anomalías debidas al ruido o valores atípicos (outliers). Un modelo sobre ajustado tiene un rendimiento muy pobre en los datos no vistos a pesar de que ofrece un rendimiento impresionante en los datos de entrenamiento. Esto se puede evitar mediante la poda, la cual es un proceso que elimina las ramas del árbol para una mejor clasificación de los elementos, sin embargo este proceso no se aplico en el respectivo clasificador.

In [17]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)
tree_pred = clf_tree.predict(X_test)

### Reporte de Clasificación

In [18]:
print(classification_report(y_test, tree_pred))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94        17
           1       0.94      0.98      0.96        51
           2       0.90      0.90      0.90        10
           3       1.00      0.60      0.75         5
           4       0.79      0.92      0.85        12
           5       0.96      0.96      0.96        26

    accuracy                           0.93       121
   macro avg       0.93      0.87      0.89       121
weighted avg       0.94      0.93      0.93       121



### Matríz de Confusión 

In [19]:
print(confusion_matrix(y_test, tree_pred))

[[15  2  0  0  0  0]
 [ 0 50  0  0  1  0]
 [ 0  0  9  0  1  0]
 [ 0  1  0  3  0  1]
 [ 0  0  1  0 11  0]
 [ 0  0  0  0  1 25]]


### Precisión de Clasificador Arbol

In [20]:
print(accuracy_score(y_test, tree_pred))

0.9338842975206612


Podemos notar que a diferencia del clasificador Naive Bayes se obtuvo una mayor precisión incluso sin utilizar el procedimiento de poda de árbol. Así mismo la diferencia de precisión entre ambos clasificadores es considerable.

## K-Nearest Neighbors

k-Nearest Neighbour por sus siglas KNN es un algoritmo de aprendizaje lento que almacena todas las instancias correspondientes a puntos de datos de entrenamiento en espacio n-dimensional. Cuando se reciben datos discretos desconocidos, analiza el número k más cercano de instancias guardadas también identificados como: "vecinos más cercanos" y devuelve la clase más común como la predicción y para los datos con valor real devuelve la media de k vecinos más cercanos.

In [21]:
from sklearn.neighbors import KNeighborsClassifier
num_vecinos = int(input("Numero de Vecinos K = "))
clf_knn = KNeighborsClassifier(n_neighbors = 5)
clf_knn.fit(X_train, y_train)
knn_pred = clf_knn.predict(X_test)

Numero de Vecinos K = 10


### Reporte de Clasificación

In [22]:
print(classification_report(y_test, knn_pred))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        17
           1       0.96      0.98      0.97        51
           2       1.00      0.80      0.89        10
           3       1.00      1.00      1.00         5
           4       0.85      0.92      0.88        12
           5       0.93      0.96      0.94        26

    accuracy                           0.95       121
   macro avg       0.96      0.93      0.94       121
weighted avg       0.95      0.95      0.95       121



### Matríz de Confusión 

In [23]:
print(confusion_matrix(y_test, knn_pred))

[[16  1  0  0  0  0]
 [ 0 50  0  0  0  1]
 [ 0  1  8  0  1  0]
 [ 0  0  0  5  0  0]
 [ 0  0  0  0 11  1]
 [ 0  0  0  0  1 25]]


### Precisión de Clasificador KNN

In [24]:
print(accuracy_score(y_test, knn_pred))

0.9504132231404959


Podemos notar que de los tres clasificadores seleccionados KNN es el que obtuvo un mayor grado de precisión para determinar la clase de los elementos en nuestro dataset, esto es aún más evidente si observamos la matriz de confusión, donde únicamente cometió dos errores, por lo que podemos observar la eficiencia de este algoritmo en el caso particular de K = 10.

# Conclusión

Tras haber experimentado con los distintos tipos de clasificadores se pudo observar que estos varían en su desempeño y por en de en su precisión para determinar la clase de algún elemento, esto nos permite deducir el por que de los métodos de ensamble, al utilizar diferentes algoritmos de clasificación en conjunto nos permite obtener un mejor desempeño en la predicción, algo que sería distinto de manera individual.

# Cuarta Entrega 
## Ensamble de Clasificadores

Como se había explicado en la anterior entrega, un ensamble es un conjunto de clasificadores que nos permite combinar múltiples hipótesis para formar una hipótesis mejor y más precisa, esto nos permitirá tener una predicción mas certera. 

Existen varios métodos de ensamble, entre los más conocidos y utilizados se encuentran los siguientes:

* **Bagging:** Mejora la estabilidad y precisión de algoritmos de aprendizaje automático
* **Adaboost:** Propone entrenar iterativamente una serie de clasificadores base, de tal modo que cada nuevo clasificador preste mayor atención a los datos clasificados erróneamente por los clasificadores anteriores
* **Random Forest:** Este método se encarga de crear y combinar diversos árboles haciendo que trabajen en conjunto de tal manera que la salida de cada uno se contará como "un voto" y la opción más votada será la respuesta del "Random Forest"

En nuestro caso particular nos enfocaremos en el método de votación. Este método toma una lista de diferentes estimadores como argumentos y un método de votación, ya sea una votación dura o suave. 
El método de votación dura utiliza las etiquetas predichas y un sistema de reglas por mayoría, mientras que el método de votación suave predice las etiquetas de clase en función de las probabilidades predichas para el clasificador.

## Promedio de Predicciones

Antes de entrar al clasificador de votación, cubramos un método muy rápido y fácil el cual se basa en el promedio de las predicciones. Simplemente sumamos los diferentes valores obtenidos de nuestros clasificadores elegidos y luego los dividimos por el número total de clasificadores, usando la división de piso para obtener un valor completo.

In [25]:
avg_preds = (knn_pred + tree_pred + bayes_pred)//3
acc = accuracy_score(y_test, avg_preds)
print("Promedio de Clasificadores: " + str(acc))

Promedio de Clasificadores: 0.7933884297520661


Podemos notar que guardamos las predicciones de los clasificadores anteriormente mostrados en variables individuales para posteriormente utilizar esos valores.

## Método de Votación

Creamos nuestro clasificador de votación a partir de los clasificadores KNN, Bayes y Árbol de Decisión mostrados anteriormente y se determina el tipo de votación, el cual en nuestro caso será una votación dura.
Al igual que con los clasificadores anteriores se le brindarán datos de entrenamiento para posteriormente hacer una predicción

In [26]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators= [('Bayes', clf_bayes), ('Tree', clf_tree), ('KNN', clf_knn)], voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('Bayes',
                              GaussianNB(priors=None, var_smoothing=1e-09)),
                             ('Tree',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                

Podemos notar como nuestro ensamble describe las características especificas de cada clasificador con el que está trabajando.

Posteriormente se calcula la precisión de predicción de nuestro ensamble

## Predicción y Resultados

In [27]:
preds = voting_clf.predict(X_test)
acc = accuracy_score(y_test, preds)

print("Precisión: " + str(acc))

Precisión: 0.9421487603305785


## Conclusión

El método de ensamble por votación es un uno de los métodos más populares y como podemos ver es un método más complejo a diferencia de un promedio simple de los resultados de clasificación, ya que como podemos observar su precisión es más elevada y por lo tanto más óptima. El ensamble nos permite obtener una mejor predicción, combinando diferentes clasificadores para obtener datos más precisos.